In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# 5.1 层和块

In [6]:
X = torch.rand(2, 20)

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0442,  0.1234, -0.3915,  0.0074, -0.1018, -0.0636,  0.1504,  0.0128,
         -0.3646, -0.1830],
        [ 0.0714,  0.2431, -0.2985,  0.0140, -0.1208, -0.0217, -0.0212,  0.0935,
         -0.2427, -0.1419]], grad_fn=<AddmmBackward0>)

#### 控制流

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)

        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.3365, grad_fn=<SumBackward0>)

#### 嵌套

In [ ]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0533, grad_fn=<SumBackward0>)

# 5.2 参数管理

- 访问参数，用于调试、诊断和可视化；
- 参数初始化；
- 在不同模型组件间共享参数。

In [12]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0072],
        [ 0.0766]], grad_fn=<AddmmBackward0>)

## 5.2.1. 参数访问

In [36]:
print(net[2].state_dict())
print(type(net[2].bias.data), type(net[2].bias))
print(net[2].state_dict()['bias'] == net[2].bias.data, net[2].bias.data)
print(net[2].bias)
print(net[2].weight.grad == None)

OrderedDict([('weight', tensor([[ 0.1599, -0.3518,  0.2931,  0.2840, -0.3372, -0.1144, -0.0933,  0.0117]])), ('bias', tensor([0.0932]))])
<class 'torch.Tensor'> <class 'torch.nn.parameter.Parameter'>
tensor([True]) tensor([0.0932])
Parameter containing:
tensor([0.0932], requires_grad=True)
True


In [32]:
print([(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
net.state_dict()['2.bias'] == net.state_dict()['2.bias'].data == net[2].state_dict()['bias']

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


tensor([True])

In [46]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet)
rgnet[0][1][0].bias.data

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


tensor([ 0.3891,  0.0506,  0.2601, -0.4070,  0.4210, -0.0776, -0.4618, -0.3798])

## 5.2.2. 参数初始化

In [ ]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data, net[0].bias.data

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


(tensor([[ 0.0257, -0.0126, -0.0033,  0.0173],
         [-0.0065,  0.0118, -0.0001, -0.0121],
         [-0.0046, -0.0063, -0.0139, -0.0028],
         [-0.0062,  0.0110,  0.0066,  0.0149],
         [ 0.0096,  0.0086,  0.0038,  0.0007],
         [ 0.0125,  0.0004,  0.0136, -0.0108],
         [ 0.0019, -0.0095, -0.0109, -0.0123],
         [ 0.0046, -0.0133, -0.0047, -0.0120]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [53]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data, net[0].bias.data

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [54]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5876, -0.0147, -0.3643,  0.2435])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [61]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.2204, -8.2161, -8.3685,  8.4781],
        [ 6.6144,  0.0000,  0.0000, -6.1535]], grad_fn=<SliceBackward0>)

In [77]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  7.7839,  7.6315, 24.4781])

## 5.2.3. 参数绑定


In [86]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 5.2.4. 小结

- 我们有几种方法可以访问、初始化和绑定模型参数。

In [92]:
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)


In [ ]:
# 访问
print(net[0].weight.data, '\n', net[0].bias.data)

print(net[0].state_dict()['weight'], '\n', net[0].state_dict()['bias'])

for name, param in net[0].named_parameters():
    print(name, param)

tensor([[ 0.1398,  0.4255, -0.3753, -0.4328],
        [-0.1223,  0.2621, -0.1239, -0.3444],
        [ 0.1512, -0.2846, -0.3581,  0.3461],
        [ 0.1768, -0.3991,  0.2120,  0.1912],
        [ 0.1712, -0.1043, -0.4592, -0.3034],
        [ 0.4076,  0.2136,  0.3750, -0.0874],
        [ 0.1358,  0.3188, -0.2352,  0.4906],
        [ 0.4113, -0.1312, -0.1755, -0.2966]]) 
 tensor([-0.1001,  0.3579,  0.3047, -0.4429,  0.3814,  0.4992, -0.1454, -0.4104])
tensor([[ 0.1398,  0.4255, -0.3753, -0.4328],
        [-0.1223,  0.2621, -0.1239, -0.3444],
        [ 0.1512, -0.2846, -0.3581,  0.3461],
        [ 0.1768, -0.3991,  0.2120,  0.1912],
        [ 0.1712, -0.1043, -0.4592, -0.3034],
        [ 0.4076,  0.2136,  0.3750, -0.0874],
        [ 0.1358,  0.3188, -0.2352,  0.4906],
        [ 0.4113, -0.1312, -0.1755, -0.2966]]) 
 tensor([-0.1001,  0.3579,  0.3047, -0.4429,  0.3814,  0.4992, -0.1454, -0.4104])
weight Parameter containing:
tensor([[ 0.1398,  0.4255, -0.3753, -0.4328],
        [-0.1223,  0.

In [97]:
# 初始化
net[0].weight.data[:] = 1
print(net[0].weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_42)
print(net[0].weight)


Parameter containing:
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)
Parameter containing:
tensor([[42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.]], requires_grad=True)


# 5.4 自定义层

In [10]:
class Mylinear(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(input, output))
        self.bias = nn.Parameter(torch.randn(output, ))
    
    def forward(self, x):
        return torch.mm(x, self.weight) + self.bias

X = torch.randn(3, 28*28)
model = Mylinear(28*28, 10)
print(model.weight.data)
model(X)

tensor([[ 0.6221,  0.4359,  0.3277,  ..., -0.7431,  1.1807,  2.2402],
        [ 0.4653, -0.0696, -1.4918,  ..., -0.7265, -0.2732, -1.7569],
        [-0.8623, -0.3617, -2.4623,  ..., -0.1959, -0.3894,  0.5762],
        ...,
        [-0.1106,  1.7827,  0.8795,  ..., -2.0277,  1.6071,  0.0746],
        [ 1.0450, -0.5560, -2.0071,  ..., -1.1902,  0.0043,  1.0860],
        [-1.1276, -1.5355, -0.8339,  ...,  0.0780,  1.4438, -0.3149]])


tensor([[ 28.6589,  -1.4791,  35.5858,   4.4783,  -0.7155, -34.6641,  -9.4101,
          28.1011, -14.2645,  -7.7370],
        [  9.4599,  -0.3365,   9.6549,  39.4441,  10.1339,  -8.9533,   1.5531,
          -2.9039, -27.1742, -23.9092],
        [  4.3896,  -0.9137,  34.6361, -53.7423,  26.1733,  19.3134, -31.6040,
         -58.0925, -33.9551, -22.3526]], grad_fn=<AddBackward0>)

In [11]:
net = nn.Sequential(Mylinear(64, 8), Mylinear(8, 1))
net(torch.rand(2, 64))

tensor([[-15.4149],
        [-24.6522]], grad_fn=<AddBackward0>)

## 5.4.4. 练习

In [ ]:
import torch
import torch.nn as nn

class Down(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(output_dim, input_dim, input_dim))
    
    def forward(self, X):
        XXT = torch.mm(X, X.T)  # shape: (input_dim, input_dim)
    
        result = torch.sum(self.weight * XXT, dim=(1, 2))
        # torch.einsum('ijk,jk->i', self.weight, XXT)
        return result

net = Down(20, 10)
X = torch.randn(20, 20)
print(net(X).shape)  # 输出: torch.Size([10])

torch.Size([10])


# 5.5. 读写文件

## 5.5.2. 加载和保存模型参数

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
torch.save(net.state_dict(), 'mlp.params')

clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))

all(torch.equal(p1, p2) for p1, p2 in zip(net.parameters(), clone.parameters()))


True